In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from dlibWrapper import dlibWrapper, load_image_from_web_link

In [ ]:
def euclidean_distance(vec1, vec2):
    return np.linalg.norm(vec1 - vec2)

dl = dlibWrapper()

def load_and_encode(image_url):
    img = load_image_from_web_link(image_url)
    embeddings = dl.detect_and_encode_faces(img)
    return embeddings[0]

In [ ]:
df = pd.read_parquet("image_links.parquet")
df = df.drop_duplicates('links').reset_index(drop=True)
filter_df = df.iloc[:20]

In [ ]:
filter_df.loc[:, 'embeddings'] = filter_df.loc[:, 'image_links'].apply(lambda x: load_and_encode(x))

In [ ]:
img1 = load_image_from_web_link("https://img.a.transfermarkt.technology/portrait/big/198589-1602167259.png?lm=1")
img2 = load_image_from_web_link("https://t1.gstatic.com/licensed-image?q=tbn:ANd9GcSKKBbXzUcl6REIpRLXbzXpqkyzEtvLEYfz46uQGH6OsJZe1xNNAXD68kDGPlqOT0oN")

In [ ]:
boxes1 = dl.face_detector(img1)
boxes2 = dl.face_detector(img2)

In [ ]:
img_shapes1 = dl.shape_predictor(img1, boxes1)
img_shapes2 = dl.shape_predictor(img2, boxes2)

In [ ]:
aligned_faces1 = dl.face_aligning(img1, img_shapes1)
aligned_faces2 = dl.face_aligning(img2, img_shapes2)


In [ ]:
fig, axes = plt.subplots(1, 2) 
axes[0].imshow(aligned_faces1[0])
axes[1].imshow(aligned_faces2[0])

In [ ]:
embeddings1 = dl.face_encoder(aligned_faces1)
embeddings2 = dl.face_encoder(aligned_faces2)
euclidean_distance(embeddings1[0], embeddings2[0])

In [ ]:
filter_df.loc[:, 'euclidean_distance'] = filter_df.loc[:, 'embeddings'].apply(lambda x: euclidean_distance(embeddings2[0], x))
filter_df[['name', 'euclidean_distance']].sort_values('euclidean_distance')